In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q flwr["simulation"] tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.2 

In [3]:
import flwr as fl
import tensorflow as tf
import numpy as np
from typing import List, Tuple
from flwr.common import Metrics
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
train_data = np.load("/content/drive/MyDrive/federated_learning_ecg/X_train.npy")
train_label = np.load("/content/drive/MyDrive/federated_learning_ecg/y_train.npy")
test_data = np.load("/content/drive/MyDrive/federated_learning_ecg/X_test.npy")
test_label = np.load("/content/drive/MyDrive/federated_learning_ecg/y_test.npy")

In [5]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(6):
  x_train.append(train_data[train_data.shape[0]//6*i:train_data.shape[0]//6*(i+1)])
  y_train.append(train_label[train_label.shape[0]//6*i:train_label.shape[0]//6*(i+1)])
  x_test.append(test_data[test_data.shape[0]//6*i:test_data.shape[0]//6*(i+1)])
  y_test.append(test_label[test_label.shape[0]//6*i:test_label.shape[0]//6*(i+1)])

In [6]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train_cid, y_train_cid, x_test_cid, y_test_cid) -> None:
        self.model = model
        self.x_train_cid, self.y_train_cid = x_train_cid, y_train_cid
        self.x_test_cid, self.y_test_cid = x_test_cid, y_test_cid

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train_cid, self.y_train_cid, epochs=5, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.x_train_cid), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x_test_cid, self.y_test_cid, verbose=0)
        return loss, len(self.x_test_cid), {"accuracy": acc}

In [7]:
def client_fn(cid: str) -> fl.client.Client:

    model = Sequential([
      layers.Conv1D(16, 3, padding='same', activation='relu', input_shape=(180,1)),
      layers.MaxPool1D(),
      layers.Conv1D(32, 3, padding='same', activation='relu'),
      layers.MaxPool1D(),
      layers.Conv1D(64, 3, padding='same', activation='relu'),
      layers.MaxPool1D(),
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(5 ,activation="softmax")
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

    x_train_cid = x_train[int(cid)]
    y_train_cid = y_train[int(cid)]

    x_test_cid = x_test[int(cid)]
    y_test_cid = y_test[int(cid)]


    return FlowerClient(model, x_train_cid, y_train_cid, x_test_cid, y_test_cid)

In [8]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:

    print(metrics)
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]


    return {"accuracy": sum(accuracies) / sum(examples)}

In [9]:
#created for initial parameters
model = Sequential([
  layers.Conv1D(16, 3, padding='same', activation='relu', input_shape=(180,1)),
  layers.MaxPool1D(),
  layers.Conv1D(32, 3, padding='same', activation='relu'),
  layers.MaxPool1D(),
  layers.Conv1D(64, 3, padding='same', activation='relu'),
  layers.MaxPool1D(),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(5 ,activation="softmax")
])

In [10]:
strategy = fl.server.strategy.FedAdagrad(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=6,
    min_evaluate_clients=3,
    min_available_clients=6,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
    evaluate_metrics_aggregation_fn=weighted_average,
)

client_resources = None


fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=6,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-15 16:14:51,484 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-15 16:14:54,220	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-15 16:14:58,910 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'memory': 7780486350.0, 'object_store_memory': 3890243174.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'memory': 7780486350.0, 'object_store_memory': 3890243174.0}
INFO flwr 2023-06-15 16:14:58,921 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-15 16:14:58,926 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-06-15 16:14:58,932 | server.py:88 | Evaluating initial para

[(5473, {'accuracy': 0.8359218239784241}), (5473, {'accuracy': 0.8359218239784241}), (5473, {'accuracy': 0.8505390286445618})]


DEBUG flwr 2023-06-15 16:22:28,233 | server.py:232 | fit_round 2 received 6 results and 0 failures
DEBUG:flwr:fit_round 2 received 6 results and 0 failures
DEBUG flwr 2023-06-15 16:22:28,309 | server.py:168 | evaluate_round 2: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 2: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 16:22:33,952 | server.py:182 | evaluate_round 2 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 2 received 3 results and 0 failures
DEBUG flwr 2023-06-15 16:22:33,955 | server.py:218 | fit_round 3: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 3: strategy sampled 6 clients (out of 6)


[(5473, {'accuracy': 0.4492965340614319}), (5473, {'accuracy': 0.44472867250442505}), (5473, {'accuracy': 0.44564223289489746})]


DEBUG flwr 2023-06-15 16:26:10,156 | server.py:232 | fit_round 3 received 6 results and 0 failures
DEBUG:flwr:fit_round 3 received 6 results and 0 failures
DEBUG flwr 2023-06-15 16:26:10,239 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 16:26:15,153 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
DEBUG flwr 2023-06-15 16:26:15,157 | server.py:218 | fit_round 4: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 4: strategy sampled 6 clients (out of 6)


...

DEBUG flwr 2023-06-15 16:48:06,085 | server.py:232 | fit_round 9 received 6 results and 0 failures
DEBUG:flwr:fit_round 9 received 6 results and 0 failures
DEBUG flwr 2023-06-15 16:48:06,138 | server.py:168 | evaluate_round 9: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 9: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 16:48:10,923 | server.py:182 | evaluate_round 9 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 9 received 3 results and 0 failures
DEBUG flwr 2023-06-15 16:48:10,930 | server.py:218 | fit_round 10: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 10: strategy sampled 6 clients (out of 6)


[(5473, {'accuracy': 0.9742371439933777}), (5473, {'accuracy': 0.9778914451599121}), (5473, {'accuracy': 0.9762470126152039})]


DEBUG flwr 2023-06-15 16:51:45,585 | server.py:232 | fit_round 10 received 6 results and 0 failures
DEBUG:flwr:fit_round 10 received 6 results and 0 failures
DEBUG flwr 2023-06-15 16:51:45,634 | server.py:168 | evaluate_round 10: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 10: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 16:51:49,415 | server.py:182 | evaluate_round 10 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 3 results and 0 failures
INFO flwr 2023-06-15 16:51:49,422 | server.py:147 | FL finished in 2210.43619167
INFO:flwr:FL finished in 2210.43619167
INFO flwr 2023-06-15 16:51:49,431 | app.py:218 | app_fit: losses_distributed [(1, 4.063242753346761), (2, 2.7957610289255777), (3, 2.560797135035197), (4, 3.1362674236297607), (5, 1.3494621117909749), (6, 1.8178023497263591), (7, 0.8564382791519165), (8, 0.43449578682581586), (9, 0.17192700505256653), (10, 0.18773825466632843)]
INFO:flwr:app_fit: losses_distributed [(1, 4.0

[(5473, {'accuracy': 0.9556002020835876}), (5473, {'accuracy': 0.9545039534568787}), (5473, {'accuracy': 0.9556002020835876})]


History (loss, distributed):
	round 1: 4.063242753346761
	round 2: 2.7957610289255777
	round 3: 2.560797135035197
	round 4: 3.1362674236297607
	round 5: 1.3494621117909749
	round 6: 1.8178023497263591
	round 7: 0.8564382791519165
	round 8: 0.43449578682581586
	round 9: 0.17192700505256653
	round 10: 0.18773825466632843
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.8407942255338033), (2, 0.4465558131535848), (3, 0.8431695103645325), (4, 0.42249831557273865), (5, 0.9193008144696554), (6, 0.5941896637280782), (7, 0.9324563145637512), (8, 0.8849503596623739), (9, 0.9761252005894979), (10, 0.9552347858746847)]}